# Creating a TPT report in a Panda dataframe in python

### Imports

In [1]:
import pyodbc
from pathlib import Path
import pandas as pd
import numpy as np
from collections import OrderedDict

### Required variables

In [2]:
# database connector
connector = pyodbc.connect('driver={SQL Server};'
                           'Server=DESKTOP-RGN6M86;'
                           'Database=intranet;'
                           'Trusted_Connection=yes;')
# date
DATE = "2020-12-31"
# client's name
CLIENT = "BILMI"
# shareclass' ISIN code
ISIN = "LU1689732417"
# portfolio file
portfolio_file_name = "template_Portfolio_BILMI.xlsx"
AODB_file_name = "AO Data Base v0.8.xlsx"
BBG_file_name = "AO_Bloomberg_Template_SII.xlsx"
root_data_path = Path('C:/Users/Hugo Durand/Workspace/data')
portfolio_file_path = root_data_path / portfolio_file_name
AODB_file_path = root_data_path / AODB_file_name
BBG_file_path = root_data_path / BBG_file_name

In [3]:
root_path = Path("C:/Users/Hugo Durand/Workspace/data")
file_name = "20201231_BILMI_BIL Invest Bonds USD Corporate Investment Grade - P USD CAP_LU1689732417_original.xlsx"
    
report = pd.read_excel(root_path / file_name,
                       sheet_name="Report",
                      skiprows=2)
i=130

#report.iloc[:,i:i+10]
#report["Unnamed: 21"]
report.drop(["17b_Asset / Liability", 
             "Unnamed: 21",
             "=AB4+EG4",
             "=Z4-EG4",
             "=AF4+EI4",
             "=AD4-EI4",
             "Unnamed: 34",
             "Unnamed: 36",
             "Unnamed: 136",
             "Unnamed: 138",
             "Currency conversion Pay attention to GBp !!!!!!!!!",
             "Diff in ccy?",
             "ccy main",
             "ccy underluying"
            ], axis=1, inplace=True)

report.rename(columns={
        "4_Portfolio currency ( B )": 
            "4_Portfolio currency (B)",
        "20_Contract size for derivatives ": 
            "20_Contract size for derivatives",
        "22_Market valuation in quotation currency  ( A ) ": 
            "22_Market valuation in quotation currency (A)",
        "24_Market valuation in portfolio currency  (B) ":
            "24_Market valuation in portfolio currency (B)",
        "29_Market exposure amount for the 3rd currency in quotation currency of the underlying asset ( C )":
            "29_Market exposure amount for the 3rd currency in quotation currency of the underlying asset (C)",
        "35_Identification type for interest rate index ": 
            "35_Identification type for interest rate index",
        "58b_Nature of the TRANCHE ": 
            "58b_Nature of the TRANCHE",
        "62_Conversion factor (convertibles)/ concordance factor  / parity (options)": 
            "62_Conversion factor (convertibles) / concordance factor / parity (options)",
        "71_Quotation currency of the underlying asset ( C )": 
            "71_Quotation currency of the underlying asset (C)",
        "123_Fund CIC code ":
            "123_Fund CIC code"
        }, inplace=True)
    
report.shape

(156, 140)

### Define required fields

In [4]:
with open("../python/fields.txt", "r") as f:
    fields = f.read().splitlines()
TPT_fields = OrderedDict.fromkeys(fields)
#print(TPT_fields)
for field, access in TPT_fields.items():
    print(field, access)

1_Portfolio identifying data None
2_Type of identification code for the fund share or portfolio None
3_Portfolio name None
4_Portfolio currency (B) None
5_Net asset valuation of the portfolio or the share class in portfolio currency None
6_Valuation date None
7_Reporting date None
8_Share price None
8b_Total number of shares None
9_% cash None
10_Portfolio Modified Duration None
11_Complete SCR Delivery None
12_CIC code of the instrument None
13_Economic zone of the quotation place None
14_Identification code of the financial instrument None
15_Type of identification code for the instrument None
16_Grouping code for multiple leg instruments None
17_Instrument name None
18_Quantity None
19_Nominal amount None
20_Contract size for derivatives None
21_Quotation currency (A) None
22_Market valuation in quotation currency (A) None
23_Clean market valuation in quotation currency (A) None
24_Market valuation in portfolio currency (B) None
25_Clean market valuation in portfolio currency (B) No

### Create TPT dataframe

In [5]:
TPT_report = pd.DataFrame(columns=fields)
#TPT_report
report.columns.difference(TPT_report.columns)

Index([], dtype='object')

## 1 - Find all instruments associated to a shareclass

### 1.1 Get infos of the shareclass from database

In [6]:
shareclass_infos = pd.read_sql_query("SELECT id, code_isin, shareclass, shareclass_currency, shareclass_name, id_subfund, type_tpt " +
                                     "FROM intranet.dbo.shareclass " +
                                     "WHERE code_isin='%s'"%ISIN, connector)
shareclass_infos["id"]

0    25
Name: id, dtype: int64

In [7]:
shareclass_nav = pd.read_sql_query("SELECT shareclass_total_net_asset, "+
                                           "nav_date, "+
                                           "share_price, "+
                                           "outstanding_shares "+
                                   " FROM intranet.dbo.nav " +
                                   "WHERE id_shareclass='%s' "%shareclass_infos["id"].iloc[0] +
                                   "AND nav_date='%s'"%DATE,
                                   connector)
shareclass_nav

,shareclass_total_net_asset,nav_date,share_price,outstanding_shares
0,11791377.12,2020-12-31,183.95,64100.413


### 1.2 Get infos of the subfund the shareclass belongs to

In [8]:
subfund_id = shareclass_infos["id_subfund"].iloc[0]
subfund_infos = pd.read_sql_query("SELECT * " +
                                  "FROM intranet.dbo.subfund " +
                                  "WHERE id='%s'"%subfund_id, connector)
subfund_infos


,id,last_update,supprime,fund_cic_code,fund_issuer_code,fund_issuer_code_type,fund_issuer_sector,subfund_code,subfund_currency,subfund_name,subfund_type,valuation_day,id_fund
0,10,2020-12-18,False,XL42,549300BQB9WT2XT2WB26,1,K6430,bil_LU3964,USD,BIL Invest Bonds USD Corporate Investment Grade,None,5,10


### 1.3 Get portfolio of the subfund

/!\ /!\ /!\ until portfolios are put in database, infos must be read from excel sheets /!\ /!\ /!\

In [ ]:
#portfolio = pd.read_sql_query("SELECT * " +
#                              "FROM intranet.dbo.portfolio " +
#                              "WHERE subfund_id='%s'"%subfund_id, connector)

#### Loading required datas

In [9]:
# loading portfolio's data
portfolio = pd.read_excel(portfolio_file_path, sheet_name="bil_invest_portfolio")
NAV = pd.read_excel(portfolio_file_path, sheet_name="NAV")
TPT_base = pd.read_excel(portfolio_file_path, sheet_name="TPT BIL Invest")

### 1.4 Find all instruments associated to subfund

In [10]:
# get subfund code from database and remove prefix
subfund_code = subfund_infos["subfund_code"].iloc[0].split("_")[1]
# get all entries with corresponding subfund code
entries = TPT_base.loc[TPT_base["SubFund Code"] == subfund_code]
entries.shape
#entries.info()

(167, 65)

### 1.5 Query instruments' infos from db

/!\ /!\ /!\ until instruments are put in database, infos must be read from excel sheets /!\ /!\ /!\

In [37]:
#instruments_infos = pd.read_sql_query("SELECT * FROM intranet.dbo.instrument", connector)
#instruments_infos

In [11]:
AODB = pd.read_excel(AODB_file_path, sheet_name="DataBase")
AODB

,#,14_Identification code of the financial instrument,BBG Ticker,12_CIC code of the instrument,13_Economic zone of the quotation place,14_Identification code of the financial instrument.1,15_Type of identification code for the instrument,17_Instrument name,17b_Asset / Liability,20_Contract size for derivatives,...,SECURITY_TYP_UNDERLYING,89_Credit quality step of the underlying asset,Dynasty SICAV,Quilvest,den-am,Athymis,BIL,Pictet,update required ?,UPD
0,BBG ID,NaN,DX615,DZ547,NaN,NaN,NaN,NaN,NaN,NaN,...,DS213,NaN,yes,yes,yes,yes,yes,yes,yes,yes
1,fix,fix,fix,fix,fix,fix,fix,fix,fix,fix,...,fix,variable,yes,yes,yes,yes,yes,yes,yes,yes
2,1,DE000A18V138,JV2247650 Corp,DE21,1,DE000A18V138,1,VONOVIA 1.625% 20,A,NaN,...,NaN,NaN,no,no,no,no,no,no,no,no
3,2,DE000A1MA9K8,EJ0400550 Corp,DE21,1,DE000A1MA9K8,1,METRO 3.375% 19,A,NaN,...,NaN,NaN,no,no,no,no,no,no,no,no
4,3,DE000A1X3GC3,EJ9325857 Corp,DE21,1,DE000A1X3GC3,1,DAIMLER 1.500% 18,A,NaN,...,NaN,NaN,no,no,no,no,no,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,3895,US8807791038,TEX UN Equity,US31,2,US8807791038,1,Terex Corp,A,NaN,...,NaN,NaN,no,no,no,no,no,yes,yes,no
3896,3896,US2774321002,EMN UN Equity,US31,2,US2774321002,1,Eastman Chemical Co,A,NaN,...,NaN,NaN,no,no,no,no,no,yes,yes,no
3897,3897,US9113121068,UPS UN Equity,US31,2,US9113121068,1,United Parcel Service Inc,A,NaN,...,NaN,NaN,no,no,no,no,no,yes,yes,no
3898,3898,US4062161017,HAL UN Equity,US31,2,US4062161017,1,Halliburton Co,A,NaN,...,NaN,NaN,no,no,no,no,no,yes,yes,no


### 1.6 drop entries associated to other shareclasses

In [12]:
# get shareclass indicator
sc_indicator = shareclass_infos["shareclass"].iloc[0]

entries = entries.loc[(entries["SC indicator"].isnull()) | (entries["SC indicator"] == sc_indicator)]
entries
#entries["Valuation Label"]

,NAV Date,SubFund Code,SubFund Name,SubFund Currency,Portfolio,Accounting Service Code,Accounting Service Name,Valuation Group Code,Valuation Group Label,GTI Code,...,Net Nav Amount,% impact,Unique ID,Multiple Leg ID,Quantity.1,MV inv_ccy,MV fnd_ccy,SC indicator,LU1689729546,Accred expenses splitted
191,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,CASH,CO,Debits/credits,850,ACCRUED EXP.,NaN,...,75455236.11,-0.002087,294864_1,NaN,-1574.74,-1574.74,-1574.74,P,NaN,NaN
192,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,CASH,CO,Debits/credits,850,ACCRUED EXP.,NaN,...,75455236.11,-0.003356,290034_1,NaN,-2532.17,-2532.17,-2532.17,P,NaN,NaN
197,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,CASH,CO,Debits/credits,800,PAYABLES,NaN,...,75455236.11,-0.311661,266100_1,NaN,-235164.32,-235164.32,-235164.32,NaN,NaN,NaN
201,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,CASH,CO,Debits/credits,850,ACCRUED EXP.,NaN,...,75455236.11,-0.015254,264287_1,NaN,-11509.91,-11509.91,-11509.91,P,NaN,NaN
203,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,CASH,CO,Debits/credits,600,RECEIVABLES,NaN,...,75455236.11,-0.000007,152001_1,NaN,-5.04,-5.04,-5.04,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,SECURITY,BO,Stock market,150,BONDS,201.0,...,75455236.11,0.358816,US60687YBC21,NaN,270000.00,271002.42,271002.42,NaN,NaN,NaN
352,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,SECURITY,BO,Stock market,150,BONDS,201.0,...,75455236.11,0.497346,US42824CBD02,NaN,375000.00,375479.89,375479.89,NaN,NaN,NaN
353,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,SECURITY,BO,Stock market,150,BONDS,201.0,...,75455236.11,0.266603,US60687YBB48,NaN,200000.00,201296.88,201296.88,NaN,NaN,NaN
354,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,SECURITY,BO,Stock market,150,BONDS,220.0,...,75455236.11,1.022893,US912810SP49,NaN,825000.00,776270.44,776270.44,NaN,NaN,NaN


#### We get 156 instruments associated to the shareclass LU1689732417 which is the expected result

###  1.7 Get infos from db

In [13]:
instruments_infos_raw = AODB.loc[AODB["14_Identification code of the financial instrument"].isin(entries["Unique ID"])]
#instruments_infos_raw.info()
instruments_infos_raw.shape
instruments_infos_raw["17_Instrument name"]

2476       SCHLUMBER 3.75% 19-01.05.24
2479        AT&T INC 4.85% 19-01.03.39
2480        WESTROCK 3.75% 18-15.03.25
2482    GE CAP INTL 4.418% 16-15.11.35
2488           BOEING 3.1% 19-01.05.26
                     ...              
3493               NSCCLF 0 ¾ 12/07/25
3494             SOCGEN 1.488 12/14/26
3495                 STLD 3 ¼ 10/15/50
3496                    T 1 ⅜ 08/15/50
3658                    T 0 ⅛ 12/31/22
Name: 17_Instrument name, Length: 149, dtype: object

### 1.8 Build instruments infos

In [33]:
#"16_Grouping code for multiple leg instruments"
#"17b_Asset / Liability",
instruments_infos_list = ["12_CIC code of the instrument",
                          "13_Economic zone of the quotation place",
                          "14_Identification code of the financial instrument",
                          "15_Type of identification code for the instrument",
                          "17_Instrument name",
                          "20_Contract size for derivatives", 
                          "21_Quotation currency (A)",
                          "32_Interest rate type",
                          "33_Coupon rate",
                          "34_Interest rate reference identification",
                          "35_Identification type for interest rate index",
                          "36_Interest rate index name",
                          "37_Interest rate Margin",
                          "38_Coupon payment frequency",
                          "39_Maturity date",
                          "40_Redemption type",
                          "41_Redemption rate",
                          "42_Callable / putable",
                          "43_Call / put date",
                          "44_Issuer / bearer option exercise",
                          "45_Strike price for embedded (call/put) options",
                          "46_Issuer name",
                          "47_Issuer identification code",
                          "49_Name of the group of the issuer",
                          "50_Identification of the group",
                          "52_Issuer country",
                          "53_Issuer economic area",
                          "54_Economic sector",
                          "55_Covered / not covered",
                          "56_Securitisation",
                          "57_Explicit guarantee by the country of issue",
                          "58_Subordinated debt",
                          "58b_Nature of the TRANCHE",
                          "59_Credit quality step",
                          "60_Call / Put / Cap / Floor",
                          "61_Strike price",
                          "62_Conversion factor (convertibles)/ concordance factor/ parity (options)",
                          "63_Effective Date of Instrument",
                          "64_Exercise type",
                          "67_CIC code of the underlying asset",
                          "68_Identification code of the underlying asset",
                          "69_Type of identification code for the underlying asset",
                          "70_Name of the underlying asset",
                          "71_Quotation currency of the underlying asset ( C )",
                          "72_Last valuation price of the underlying asset",
                          "73_Country of quotation of the underlying asset",
                          "74_Economic Area of quotation of the underlying asset",
                          "75_Coupon rate of the underlying asset",
                          "76_Coupon payment frequency of the underlying asset",
                          "77_Maturity date of the underlying asset",
                          "78_Redemption profile of the underlying asset",
                          "79_Redemption rate of the underlying asset",
                          "80_Issuer name of the underlying asset",
                          "81_Issuer identification code of the underlying asset",
                          "82_Type of issuer identification code of the underlying asset",
                          "83_Name of the group of the issuer of the underlying asset",
                          "84_Identification of the group of the underlying asset",
                          "85_Type of the group identification code of the underlying asset",
                          "86_Issuer country of the underlying asset",
                          "87_Issuer economic area of the underlying asset",
                          "88_Explicit guarantee by the country of issue of the underlying asset",
                          "89_Credit quality step of the underlying asset"]

#instruments_infos = pd.DataFrame(columns=instruments_infos_list)

instruments_infos_raw['12_CIC code of the instrument']

2476    XL21
2479    US21
2480    CA21
2482    US21
2488    US21
        ... 
3493    XL21
3494    XL21
3495    DE21
3496    US11
3658    US11
Name: 12_CIC code of the instrument, Length: 149, dtype: object

In [34]:
instruments_db_infos = instruments_infos_raw[instruments_infos_list]
#instruments_db_infos.merge(instruments_infos_raw)
instruments_db_infos

,12_CIC code of the instrument,13_Economic zone of the quotation place,14_Identification code of the financial instrument,15_Type of identification code for the instrument,17_Instrument name,20_Contract size for derivatives,21_Quotation currency (A),32_Interest rate type,33_Coupon rate,34_Interest rate reference identification,...,80_Issuer name of the underlying asset,81_Issuer identification code of the underlying asset,82_Type of issuer identification code of the underlying asset,83_Name of the group of the issuer of the underlying asset,84_Identification of the group of the underlying asset,85_Type of the group identification code of the underlying asset,86_Issuer country of the underlying asset,87_Issuer economic area of the underlying asset,88_Explicit guarantee by the country of issue of the underlying asset,89_Credit quality step of the underlying asset
2476,XL21,0,US806851AJ09,1,SCHLUMBER 3.75% 19-01.05.24,NaN,USD,FIXED,3.75,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2479,US21,2,US00206RHK14,1,AT&T INC 4.85% 19-01.03.39,NaN,USD,FIXED,4.85,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2480,CA21,2,US96145DAF24,1,WESTROCK 3.75% 18-15.03.25,NaN,USD,FIXED,3.75,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2482,US21,2,US36164QNA21,1,GE CAP INTL 4.418% 16-15.11.35,NaN,USD,FIXED,4.418,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2488,US21,2,US097023CH65,1,BOEING 3.1% 19-01.05.26,NaN,USD,FIXED,3.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,XL21,0,US637639AE51,1,NSCCLF 0 ¾ 12/07/25,NaN,USD,FIXED,0.75,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3494,XL21,0,US83368RAZ55,1,SOCGEN 1.488 12/14/26,NaN,USD,VARIABLE,1.488,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3495,DE21,1,US858119BP41,1,STLD 3 ¼ 10/15/50,NaN,USD,FIXED,3.25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3496,US11,2,US912810SP49,1,T 1 ⅜ 08/15/50,NaN,USD,FIXED,1.375,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
instruments_infos["17_Instrument name"] = entries["Valuation Label"]
test = entries.rename(columns={"Valuation Label" : "17_Instrument name",
                        "Unique ID" : "14_Identification code of the financial instrument"})
test = test.loc[:,["17_Instrument name", "14_Identification code of the financial instrument", "Valuation Group Label"]].set_index(["17_Instrument name"])
#test = instruments_infos.merge(entries, left_on=["17_Instrument name"], right_on=["Valuation Label"]) 
#instruments_infos["14_Identification code of the financial instrument"] = entries["Unique ID"]

#test[["14_Identification code of the financial instrument", "17_Instrument name"]]

#test = instruments_infos.loc[:,["17_Instrument name", "14_Identification code of the financial instrument"]].merge(entries.loc[:,["Valuation Label", "Unique ID"]], left_on=["17_Instrument name"], right_on=["Valuation Label"])
instruments_infos.set_index(["17_Instrument name"], inplace=True)
instruments_infos
#instruments_infos.loc[["ADVANCE AUT 1.75% 20-01.10.27"], "14_Identification code of the financial instrument"].update(test["14_Identification code of the financial instrument"])
#instruments_infos.loc["ADVANCE AUT 1.75% 20-01.10.27", "14_Identification code of the financial instrument"]
#test.set_index(["17_Instrument name"])

#instruments_infos.reset_index(inplace=True)
#instruments_infos
#test
#test[]
#idn = "14_Identification code of the financial instrument"
#fields = instruments_infos_list[:3]
#instruments_infos.loc[instruments_infos[idn].isin(instruments_infos_raw[idn]), fields] = instruments_infos_raw[fields]

#instruments_infos[instruments_infos_list[:10]]
#for field in instruments_infos_list:
#    if field not in ['16_Grouping code for multiple leg instruments',
#                     '17_Instrument name',
#                     '14_Identification code of the financial instrument']:
#        instruments_infos.loc[
#            instruments_infos[idn].isin(
#                instruments_infos_raw[idn]),
#            field] = instruments_infos_raw[field].to_numpy()
#instruments_infos

,12_CIC code of the instrument,13_Economic zone of the quotation place,14_Identification code of the financial instrument,15_Type of identification code for the instrument,16_Grouping code for multiple leg instruments,17b_Asset / Liability,20_Contract size for derivatives,21_Quotation currency (A),32_Interest rate type,33_Coupon rate,...,80_Issuer name of the underlying asset,81_Issuer identification code of the underlying asset,82_Type of issuer identification code of the underlying asset,83_Name of the group of the issuer of the underlying asset,84_Identification of the group of the underlying asset,85_Type of the group identification code of the underlying asset,86_Issuer country of the underlying asset,87_Issuer economic area of the underlying asset,88_Explicit guarantee by the country of issue of the underlying asset,89_Credit quality step of the underlying asset
17_Instrument name,,,,,,,,,,,,,,,,,,,,,
NaN,XT72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,XT72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,XT72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,XT72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,XT72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
idx = test.index[test["Valuation Group Label"] == "ACCRUED EXP."].to_list()
idx += test.index[test["Valuation Group Label"] == "CASH"].to_list()
idx += test.index[test["Valuation Group Label"] == "F.E.T."].to_list()
idx += test.index[test["Valuation Group Label"] == "RECEIVABLES"].to_list()
idx += test.index[test["Valuation Group Label"] == "PAYABLES"].to_list()
instruments_infos.loc[idx, "12_CIC code of the instrument"] = "XT72"
frais = instruments_infos.loc[idx, "12_CIC code of the instrument"]
frais

KeyError: "None of [Index(['Subscription Tax P', 'Fixed Servicing Fee P', 'Management Fee P',\n       'C.B. Call Deposits RBC IS Bank Luxbg', 'Int.Rec.Bank Accounts',\n       'Receivable Interests on Bonds', 'Payable on Purchases'],\n      dtype='object', name='17_Instrument name')] are in the [index]"

In [21]:
CIC_instruments_in_db = instruments_infos_raw[['17_Instrument name', '12_CIC code of the instrument']]
CIC_instruments_in_db.set_index("17_Instrument name", inplace=True)
CIC_instruments_in_db

,12_CIC code of the instrument
17_Instrument name,
SCHLUMBER 3.75% 19-01.05.24,XL21
AT&T INC 4.85% 19-01.03.39,US21
WESTROCK 3.75% 18-15.03.25,CA21
GE CAP INTL 4.418% 16-15.11.35,US21
BOEING 3.1% 19-01.05.26,US21
...,...
NSCCLF 0 ¾ 12/07/25,XL21
SOCGEN 1.488 12/14/26,XL21
STLD 3 ¼ 10/15/50,DE21


In [24]:
all_CIC = frais.to_frame().append(CIC_instruments_in_db)
all_CIC

,12_CIC code of the instrument
17_Instrument name,
Subscription Tax P,XT72
Fixed Servicing Fee P,XT72
Management Fee P,XT72
C.B. Call Deposits RBC IS Bank Luxbg,XT72
Int.Rec.Bank Accounts,XT72
...,...
NSCCLF 0 ¾ 12/07/25,XL21
SOCGEN 1.488 12/14/26,XL21
STLD 3 ¼ 10/15/50,DE21


In [139]:
instruments_CIC = instruments_infos["12_CIC code of the instrument"]
instruments_CIC = instruments_CIC.to_frame()
instruments_CIC.merge(CIC_instruments_in_db, how='right', on="17_Instrument name", left_index=True, right_index=True)
instruments_CIC

,12_CIC code of the instrument
17_Instrument name,
Subscription Tax P,XT72
Fixed Servicing Fee P,XT72
Payable on Purchases,NaN
Management Fee P,XT72
Int.Rec.Bank Accounts,NaN
...,...
MIZU FIN FRN 20-25.05.24,NaN
HP ENTERPRI FRN 19-12.03.21,NaN
MIZU FIN FRN 19-13.09.23,NaN


## 2 - Fill infos of the instruments

### 1.1 Fill instruments' infos

In [142]:
for field in instruments_infos_list:
    TPT_report[field] = instruments_infos[field]
#TPT_report.iloc[:10, 10:20]

,10_Portfolio Modified Duration,11_Complete SCR Delivery,12_CIC code of the instrument,13_Economic zone of the quotation place,14_Identification code of the financial instrument,15_Type of identification code for the instrument,16_Grouping code for multiple leg instruments,17_Instrument name,17b_Asset / Liability,18_Quantity
191,NaN,NaN,NaN,NaN,294864_1,NaN,NaN,Subscription Tax P,NaN,NaN
192,NaN,NaN,NaN,NaN,290034_1,NaN,NaN,Fixed Servicing Fee P,NaN,NaN
197,NaN,NaN,NaN,NaN,266100_1,NaN,NaN,Payable on Purchases,NaN,NaN
201,NaN,NaN,NaN,NaN,264287_1,NaN,NaN,Management Fee P,NaN,NaN
203,NaN,NaN,NaN,NaN,152001_1,NaN,NaN,Int.Rec.Bank Accounts,NaN,NaN
204,NaN,NaN,NaN,NaN,151301_1,NaN,NaN,Receivable Interests on Bonds,NaN,NaN
206,NaN,NaN,NaN,NaN,144120_1,NaN,NaN,C.B. Call Deposits RBC IS Bank Luxbg,NaN,NaN
207,NaN,NaN,XL21,0,US90351DAB38,1,NaN,UBS GRP 4.125% 15-24.09.25 144A,A,NaN
208,NaN,NaN,US21,2,US49446RAV15,1,NaN,KIMCO REALT 3.70% 19-01.10.49,A,NaN
209,NaN,NaN,CA21,2,US38141EC311,1,NaN,GS 4.8% 14-08.07.44,A,NaN


## 3 - Fill infos of the shareclass  

### 3.1 Constant infos

In [143]:
TPT_report["1_Portfolio identifying data"] = ISIN
TPT_report["2_Type of identification code for the fund share or portfolio"] = shareclass_infos["type_tpt"].iloc[0]
TPT_report["3_Portfolio name"] = shareclass_infos["shareclass_name"].iloc[0]
TPT_report["4_Portfolio currency ( B )"] = shareclass_infos["shareclass_currency"].iloc[0]

### 3.2 NAV infos

In [147]:
TPT_report["5_Net asset valuation of the portfolio or the share class in portfolio currency"] = shareclass_nav["shareclass_total_net_asset"].iloc[0]
TPT_report["6_Valuation date"] = shareclass_nav["nav_date"].iloc[0]
TPT_report["7_Reporting date"] = DATE
TPT_report["8_Share price"] = shareclass_nav["share_price"].iloc[0]
TPT_report["8b_Total number of shares"] = shareclass_nav["outstanding_shares"].iloc[0]
#TPT_report

## 4 - Fill fund & subfund infos

###  4.1 subfund infos

In [157]:
TPT_report["115_Fund Issuer Code"] = subfund_infos["fund_issuer_code"].iloc[0]
TPT_report["116_Fund Issuer Code Type"] = subfund_infos["fund_issuer_code_type"].iloc[0] 
TPT_report["117_Fund Issuer Name"] = subfund_infos["subfund_name"].iloc[0]
TPT_report["118_Fund Issuer Sector"] = subfund_infos["fund_issuer_sector"].iloc[0] 
TPT_report["123_Fund CIC code "] = subfund_infos["fund_cic_code"].iloc[0]
# subfund_code subfund_currency 

### 4.2 fund infos

In [150]:
# get infos of fund from db
fund_id = subfund_infos["id_fund"].iloc[0]
fund_infos = pd.read_sql_query("SELECT * " +
                                  "FROM intranet.dbo.fund " +
                                  "WHERE id='%s'"%fund_id, connector)
fund_infos

,id,last_update,supprime,custodian_name,fund_code,fund_currency,fund_custodian_country,fund_issuer_country,fund_issuer_group_code,fund_issuer_group_code_type,fund_issuer_group_name,fund_name,fund_type,id_client
0,10,2020-12-18,False,RBC Luxembourg,BIL1,EUR,LU,LU,549300OKO8371SPM7H13,1,BIL Invest,BIL Invest,UCITS,3


In [158]:
TPT_report["119_Fund Issuer Group Code"] = fund_infos["fund_issuer_group_code"].iloc[0]
TPT_report["120_Fund Issuer Group Code Type"] = fund_infos["fund_issuer_group_code_type"].iloc[0]
TPT_report["121_Fund Issuer Group name"] = fund_infos["fund_issuer_group_name"].iloc[0]
TPT_report["122_Fund Issuer Country"] = fund_infos["fund_issuer_country"].iloc[0]
TPT_report["123a_Fund Custodian Country"] = fund_infos["fund_custodian_country"].iloc[0]
TPT_report["133_custodian_name"] = fund_infos["custodian_name"].iloc[0]

#TPT_report.iloc[:,125:135]

## 4 - Fill Bloomberg infos 

/!\ /!\ /!\ until Bloomberg infos are put in database, infos must be read from excel sheets /!\ /!\ /!\

In [161]:
#BBG = pd.read_sql_query("SELECT * FROM intranet.dbo.bloomberg", connector)
#BBG

In [172]:
BBG = pd.read_excel(BBG_file_path, sheet_name="Hard Copy", skiprows=2)
BBG

,Fund,ISIN,BBG ticker,Instrument name,YAS_RISK,YAS_MOD_DUR,CV_MODEL_DELTA_S,DUR_ADJ_MTY_MID,MTY_TYP,credit_sensitivity,cv_model_gamma_v,CV_MODEL_vega,cv_model_cnvs_prem,cv_model_bond_val,Unnamed: 14
0,PLACEURO BG OPPORTU,FR0013153160,JK6295154 Corp,"LAGARDERE 2,75 16-23",2.32445,2.30318,-,2.3128,CALLABLE,NaN,-,-,-,-,-
1,PLACEURO BG OPPORTU,FR0013489739,FR0013489739 Corp,"KORIAN 0,875 20-27",3.11847,5.8303,23.199,6.01875,CONVERTIBLE,NaN,0.00505712,0.228387,3.02877,49.621,46.5922
2,PLACEURO BG OPPORTU,XS1562036704,AM3527430 Corp,MARCOLIN FRN 17-23,0.0388041,0.0425853,-,0.0440292,CALLABLE,NaN,-,-,-,-,-
3,PLACEURO BG OPPORTU,XS1600514696,XS1600514696 Corp,"BURGERK 6,00 17-24",0.0238275,0.0198265,-,3.045,CALLABLE,NaN,-,-,-,-,-
4,PLACEURO BG OPPORTU,XS2167595672,XS2167595672 Corp,"EUROFINS 3,75 20-26",5.91718,4.92724,-,5.212,CALLABLE,NaN,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,NaN,XS2138128314,XS2138128314 Corp,NaN,1.39905,1.2641,-,4.98248,CALLABLE,NaN,-,-,-,-,-
997,NaN,XS2178833773,XS2178833773 Corp,NaN,5.05458,4.35226,-,4.58681,CALLABLE,NaN,-,-,-,-,-
998,NaN,XS2187689380,XS2187689380 Corp,NaN,7.85952,7.09712,-,7.10421,PERP/CALL,NaN,-,-,-,-,-
999,NaN,XS2198388592,XS2198388592 Corp,NaN,1.59393,1.36661,-,3.59128,CALLABLE,NaN,-,-,-,-,-


In [173]:
instruments_BBG_infos = BBG.loc[BBG["ISIN"].isin(entries["Unique ID"])]
instruments_BBG_infos

,Fund,ISIN,BBG ticker,Instrument name,YAS_RISK,YAS_MOD_DUR,CV_MODEL_DELTA_S,DUR_ADJ_MTY_MID,MTY_TYP,credit_sensitivity,cv_model_gamma_v,CV_MODEL_vega,cv_model_cnvs_prem,cv_model_bond_val,Unnamed: 14
697,NaN,US90351DAB38,US90351DAB38 Corp,NaN,4.98514,4.31614,-,4.31149,AT MATURITY,NaN,-,-,-,-,-
698,NaN,US49446RAV15,US49446RAV15 Corp,NaN,18.9306,17.6504,-,17.6289,CALLABLE,NaN,-,-,-,-,-
699,NaN,US38141EC311,US38141EC311 Corp,NaN,21.3661,15.156,-,15.1098,CALLABLE,NaN,-,-,-,-,-
700,NaN,US49446RAX70,US49446RAX70 Corp,NaN,6.71292,6.50225,-,6.63674,CALLABLE,NaN,-,-,-,-,-
701,NaN,US911312BW51,US911312BW51 Corp,NaN,26.718,17.3217,-,17.1246,CALLABLE,NaN,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,NaN,US60687YBC21,US60687YBC21 Corp,NaN,0.156501,0.155861,-,0.155962,CALLABLE,NaN,-,-,-,-,-
842,NaN,US42824CBD02,US42824CBD02 Corp,NaN,0.197453,0.197121,-,0.197006,AT MATURITY,NaN,-,-,-,-,-
843,NaN,US60687YBB48,US60687YBB48 Corp,NaN,0.208708,0.206846,-,0.206477,CALLABLE,NaN,-,-,-,-,-
844,NaN,US912810SP49,US912810SP49 Corp,NaN,22.4108,23.877,-,23.6321,NORMAL,NaN,-,-,-,-,-


In [ ]:
TPT_report["90_Modified Duration to maturity date"] = instruments_BBG_infos
TPT_report["91_Modified duration to next option exercise date"]
TPT_report["92_Credit sensitivity"]
TPT_report["93_Sensitivity to underlying asset price (delta)"]
TPT_report["94_Convexity / gamma for derivatives"]
TPT_report["94b_Vega"]
TPT_report["127_Bond Floor (convertible instrument only)"]
TPT_report["128_Option premium (convertible instrument only)"]